In [61]:
import scqubits as scq
from parametersweep_eg import ParameterSweep_EG
from parametersweep_ui_elements import ParameterSweep_UI_Elements
import numpy as np
from scipy import interpolate
import random

In [62]:
parametersweep_main = ParameterSweep_EG()
parametersweep = parametersweep_main.parametersweep

Bare spectra:   0%|          | 0/21 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/21 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/21 [00:00<?, ?it/s]

Dressed spectrum:   0%|          | 0/21 [00:00<?, ?it/s]

In [63]:
def create_point(sweep, current_values, sweep_value, evals_value):
    point = []
    for param, value in current_values.items():
        if param == sweep:
            point.append(sweep_value)
        else:
            point.append(value)
    point.append(evals_value)

    return point

def create_points(parametersweep, evals):
    points = ()
    for param_x in parametersweep.param_info.values():
        points = points + (param_x,)

    evals_x = np.linspace(0, evals-1, evals)
    points = points + (evals_x,)

    return points

def get_interpolated_data_size(parametersweep, sweep, evals):
    interpolated_size = ()
    if len(parametersweep.param_info.keys()) == 1:
            interpolated_size = interpolated_size + (list(parametersweep.param_info.values())[0].size,)
    else:
        for param, param_x in parametersweep.param_info.items():
            if param != sweep:
                interpolated_size = interpolated_size + (param_x.size,)

    interpolated_size = interpolated_size + (evals,)

    return interpolated_size

def create_interpolated_data(parametersweep, sweep, current_values):
    evals = parametersweep.eigenvals().shape[-1]
    sweep_x = parametersweep.param_info[sweep]
    points = create_points(parametersweep, evals)
    values = parametersweep["evals"]

    matrix_size = get_interpolated_data_size(
        parametersweep, sweep, evals)
    matrix = np.zeros(matrix_size)
    row_index = 0
    for sweep_value in sweep_x:
        column_index = 0
        for evals_value in points[-1]:
            point = create_point(
                sweep, current_values, sweep_value, evals_value)
            matrix[row_index, column_index] = interpolate.interpn(
                points, values, np.array(point))[0]
            column_index += 1
        row_index += 1
    return matrix

In [65]:
interpolated_data = create_interpolated_data(parametersweep, "flux", {"flux":0.5})

In [73]:
interpolated_data

array([[-48.9776994 , -45.02697343, -44.3665604 , -41.17760303,
        -41.16622697],
       [-46.20093903, -42.36845184, -41.61700884, -38.63990159,
        -38.59686951],
       [-38.24067872, -34.80636811, -33.70287551, -31.50027076,
        -31.23467726],
       [-26.2464243 , -23.55102773, -21.74176517, -21.00420216,
        -20.28030703],
       [-12.92358598, -11.1616125 ,  -9.5957706 ,  -8.64566456,
         -8.35943018],
       [ -6.99827928,  -4.69531953,  -4.62552596,  -2.63768756,
         -2.48304931],
       [-18.87570623, -15.64794849, -14.58995824, -14.2886359 ,
        -12.55514909],
       [-31.98280277, -28.24860445, -27.41933043, -26.01659179,
        -24.62651387],
       [-41.71561812, -37.77766933, -37.10906522, -34.70960396,
        -33.93136215],
       [-46.7181691 , -42.81876292, -42.12035163, -39.11409862,
        -39.01603509],
       [-46.44333952, -42.84082251, -41.89037067, -39.35888627,
        -38.64324685],
       [-41.09552935, -38.11348982, -36.578

In [18]:
interpolated_data = interpolate.interp1d(parametersweep.param_info["flux"], parametersweep["evals"])

ValueError: x and y arrays must be equal in length along interpolation axis.